### Imports

In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
from datasets import load_dataset 
from CodeSyntaxConcept.tokenizer import CodeTokenizer
import CodeSyntaxConcept.utils as utils
from statistics import mean, median
import json

### Parameters

In [2]:
checkpoint = "EleutherAI/gpt-neo-2.7B"
file_path = "output/aggregation_function/out_codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv"
language = "python"
aggregates_path = "output/aggregation_function/" + file_path.split('/')[1]

### Tokenizer

In [3]:
tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)

### Actual Token Predictions

In [8]:
df_actual_ntp = pd.read_csv(file_path, index_col=0)

In [9]:
df_actual_ntp.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,loss
0,"def encode_images(format_dict):\n """"""b64-en...","[('def', 'def', 'function_definition'), ('enco...","[(4299, 'def', 'function_definition'), (37773,...","[4299, 37773, 62, 17566, 7, 18982, 62, 11600, ...",355,"[('module', 0.18295365571975708), ('_', 0.4385...","[('ĠIncarn', 2.5102770068782115e-15), ('Ġenthu...","[('Ġencode', 3.8248301279963925e-05), ('_', 0....",0.990880
1,"def _process_execute_error(self, msg):\n ...","[('def', 'def', 'function_definition'), ('_pro...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 14681, 62, 41049, 62, 18224, 7, 9...",398,"[('module', 0.1829550713300705), ('Py', 0.0530...","[('ĠIncarn', 2.5107178875272574e-15), ('Ġansw'...","[('Ġ_', 0.03443761542439461), ('process', 0.00...",1.064626
2,"def long_file(data_file, dataformat, sample_li...","[('def', 'def', 'function_definition'), ('long...","[(4299, 'def', 'function_definition'), (890, '...","[4299, 890, 62, 7753, 7, 7890, 62, 7753, 11, 1...",1921,"[('module', 0.18295776844024658), ('_', 0.2431...","[('ĠIncarn', 2.510304958965471e-15), ('ĠIncarn...","[('Ġlong', 2.444034362270031e-05), ('_', 0.243...",0.959774
3,"def snapshot(self,\n label,\n ...","[('def', 'def', 'function_definition'), ('snap...","[(4299, 'def', 'function_definition'), (27479,...","[4299, 27479, 7, 944, 11, 198, 220, 220, 220, ...",801,"[('module', 0.18295776844024658), ('_', 0.4098...","[('ĠIncarn', 2.510304958965471e-15), ('TRUMP',...","[('Ġsnapshot', 2.3139917175285518e-05), ('(', ...",0.925576
4,def unload(self):\n '''Releases rendere...,"[('def', 'def', 'function_definition'), ('unlo...","[(4299, 'def', 'function_definition'), (555, '...","[4299, 555, 2220, 7, 944, 2599, 198, 220, 220,...",79,"[('module', 0.18295776844024658), ('pack', 0.1...","[('ĠIncarn', 2.510304958965471e-15), ('Ġ//[', ...","[('Ġun', 0.0002941525017376989), ('load', 0.03...",1.132517


### Token Binding

In [10]:
def bind_bpe_tokens(
    node,              #Tree sitter ast tree
    encoding,          #Token encoding
    actual_probs,      #Actual probabilities
    lines              #Source code Snippet
): 
    """Traverses the tree and bind the leaves with the corresponding node"""
    tree_node = {}
    tree_node['type'] = node.type
    tree_node['children'] = []
    tree_node['bindings'] = []

    node_span = [utils.convert_to_offset(node.start_point, lines), utils.convert_to_offset(node.end_point, lines)]
    for encoding_index, token_span in enumerate(encoding.offset_mapping):
        if (node_span[0] <= token_span[0] and token_span[0] < node_span[1]) \
        or (node_span[0] < token_span[1] and token_span[1] <= node_span[1]) \
        or (node_span[0] >= token_span[0] and token_span[1] >= node_span[1]) :
            tree_node['bindings'].append(actual_probs[encoding_index])
    
    for n in node.children:
        tree_node['children'].append(bind_bpe_tokens(n, encoding, actual_probs, lines))

    return tree_node
        

In [11]:
encoding = tokenizer.tokenizer(df_actual_ntp.iloc[0]['whole_func_string'], return_offsets_mapping=True)
assert len(eval(df_actual_ntp.iloc[0]['model_input_ids'])) == len(encoding['input_ids'])

In [12]:
binded_tree_col = []
for index, row in df_actual_ntp.iterrows():
    tree = tokenizer.parser.parse(bytes(row['whole_func_string'], "utf8"))
    encoding = tokenizer.tokenizer(row['whole_func_string'], return_offsets_mapping=True)
    actual_logits = eval(row['actual_prob_case'])
    actual_logits.insert(0,(tokenizer.tokenizer.decode(eval(row['model_input_ids'])[0]),'FIRST_TOKEN'))
    binded_tree = bind_bpe_tokens(tree.root_node, encoding, actual_logits, row['whole_func_string'].split('\n'))
    binded_tree_col.append(binded_tree)
df_actual_ntp['binded_tree'] = binded_tree_col

2023-03-02 13:59:40.473618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 13:59:43.589638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-03-02 13:59:43.589739: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-03-02 13:59:43.589749: W tensorflow/compiler/tf2tensorrt/utils/py_utils

In [13]:
def process_bindings(
    node: dict,     #Binded AST tree with actual probabilities
) -> None:
    node_actual_probs = [binding[1] for binding in node['bindings'] if isinstance(binding[1], float)]
    node['median_prob'] = node['max_prob'] = node['min_prob'] = node['avg_prob'] =  node['std'] = None
    ## len is zero if node correspond to FIRST_TOKEN = 'def' 
    if(len(node_actual_probs) > 0):
        ## BOOTSTRAPPING-> 
        node_actual_probs = utils.bootstrapping(node_actual_probs, np.mean, size=500).tolist()
        ##
        node['median_prob'] = median(node_actual_probs) 
        node['max_prob'] = max(node_actual_probs) 
        node['min_prob'] = min(node_actual_probs)
        node['avg_prob'] = mean(node_actual_probs)
        node['std'] = np.std(node_actual_probs)
    for child in node['children']:
        process_bindings(child)

In [14]:
df_actual_ntp['binded_tree'].apply(lambda binded_tree: process_bindings(binded_tree))
print(df_actual_ntp.iloc[0]['binded_tree']['children'][0]['children'][1]['median_prob'])
print(df_actual_ntp.iloc[1]['binded_tree']['children'][0]['children'][1]['median_prob'])

In [11]:
df_actual_ntp.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,binded_tree
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[('ers', 0.03083181567490101), ('_', 0.2419087...","[('anwhile', 1.03836617568492e-16), ('ousy', 3...","[('Ġget', 0.0024285861290991306), ('_', 0.2419...","{'type': 'module', 'children': [{'type': 'func..."
1,"def execute_pipeline(pipeline, environment_dic...","[('def', 'def', 'function_definition'), ('exec...","[(4299, 'def', 'function_definition'), (12260,...","[4299, 12260, 62, 79, 541, 4470, 7, 79, 541, 4...",492,"[('ers', 0.03083181567490101), ('_', 0.1853068...","[('anwhile', 1.03836617568492e-16), ('icester'...","[('Ġexecute', 5.97450380155351e-05), ('_', 0.1...","{'type': 'module', 'children': [{'type': 'func..."
2,"def _decode(self, data):\n '''\n ...","[('def', 'def', 'function_definition'), ('_dec...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 12501, 1098, 7, 944, 11, 1366, 25...",583,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('dec', 0.00133...","{'type': 'module', 'children': [{'type': 'func..."
3,"def _repr_html_(self):\n """"""\n J...","[('def', 'def', 'function_definition'), ('_rep...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 260, 1050, 62, 6494, 41052, 944, ...",221,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('re', 0.001378...","{'type': 'module', 'children': [{'type': 'func..."
4,"def build_shape(relation, nodes, ways):\n ""...","[('def', 'def', 'function_definition'), ('buil...","[(4299, 'def', 'function_definition'), (1382, ...","[4299, 1382, 62, 43358, 7, 49501, 11, 13760, 1...",454,"[('ers', 0.03083181567490101), ('_', 0.2842166...","[('anwhile', 1.03836617568492e-16), ('buquerqu...","[('Ġbuild', 0.0005856865900568664), ('_', 0.28...","{'type': 'module', 'children': [{'type': 'func..."


In [12]:
##Convert to JSON Object
df_actual_ntp['binded_tree'] = df_actual_ntp['binded_tree'].map(lambda binded_tree: json.dumps(binded_tree))

In [13]:
df_actual_ntp.to_csv(aggregates_path)